# CS378 Project

Neha Desaraju (nd9922)
ndesaraju@utexas.edu

## Robust Control

This part deals with the implementation of the paper ["Approximate Robust Control of Uncertain Dynamical Systems"](https://arxiv.org/pdf/1903.00220.pdf) (Leurent et. al). The paper describes safe control policies and methods to improve robustness to maximize the worst-case possibility with respect to a set of possible behavioral models, as described by robust control theory.

In [1]:
import gymnasium as gym
import highway_env

import torch
import torch.nn as nn
import gpytorch as gpt
import numpy as np
from collections import namedtuple
from tqdm import tqdm, trange

from scripts.record_utils import *

Because the methods we will be demonstrating involve a continuous action space, we must change the `_reward()` method in the given environment (which requires a discrete action space).

In [2]:
from highway_env.envs.roundabout_env import RoundaboutEnv
from highway_env.vehicle.controller import MDPVehicle

class ContinuousRoundaboutEnv(RoundaboutEnv):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        super().configure({
            "action": {
                "type": "ContinuousAction",
            },
            "observation": {
                "type": "Kinematics",
            }
        })

    def _rewards(self, action: np.ndarray) -> float:
        # we remove the lane_change_reward, which was `action in [0, 2]`

        return {
            "collision_reward": self.vehicle.crashed,
            "high_speed_reward": MDPVehicle.get_speed_index(self.vehicle)
            / (MDPVehicle.DEFAULT_TARGET_SPEEDS.size - 1),
            "on_road_reward": self.vehicle.on_road,
        }

First we will set up a sample environment and run through an episode with random actions.

In [3]:
env = ContinuousRoundaboutEnv(render_mode='rgb_array')
env.reset()

env, display = record_videos(env)

env.reset()
done = False
truncated = False
while not done and not truncated:
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
display.stop()
del display
env.close()
show_videos()

Moviepy - Building video /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-0.mp4


### Noisy observations

In order to simulate noisy observations so we can demonstrate methods of robustness (predicting system dynamics against noise), we will add a simple Gaussian noise to the environment. (This code is described and demonstrated in [Simple Noisy Environment Augmentation for Reinforcement Learning](https://arxiv.org/pdf/2305.02882.pdf), but we will keep a constant probability parameter of 1, so the noise is always added.)

In [4]:
class RandomNormalNoisyObservation(gym.ObservationWrapper):
    """Adds random Normal noise to the observations of the environment."""

    def __init__(self, env, loc=0.0, scale=0.00005):
        """Initializes the :class:`RandomNormalNoisyObservation` wrapper.

        Args:
            env (gym.Env): The environment to apply the wrapper
            loc (float, optional): Mean ("centre") of the noise distribution.
                Defaults to 0.0.
            scale (float, optional): Standard deviation (spread or "width") of the noise distribution.
                Must be non-negative. Defaults to 0.0001.
        """
        super().__init__(env)
        self.loc = loc
        self.scale = scale

    def observation(self, observation):
        """Returns the potentially modified observation."""
        return observation + np.random.normal(loc=self.loc, scale=self.scale, size=observation.shape)

# env = RandomNormalNoisyObservation(env, loc=0.0, scale=0.1)

### Gaussian Process Dynamics Model

In [5]:
Transition = namedtuple('Transition', ['state', 'action', 'next_state', 'safe_next_state', 'rewards'])

def collect_interaction_data(env, size=1000, min_num_safe=300, min_num_crashed=100):
    data = []
    done = True
    truncated = False

    num_safe = 0
    num_crashed = 0
    num_unsafe = 0

    with tqdm(total=size) as pbar:
        while len(data) < size:
            if done or truncated:
                previous_obs, info = env.reset()

            action = env.action_space.sample()
            obs, reward, done, truncated, info = env.step(action)
            safe_next_state = 1 if info['rewards']['on_road_reward'] and not info['rewards']['collision_reward'] else 0

            # # if there are enough not on road states, stop recording those
            # if not info['rewards']['on_road_reward']:
            #     if num_unsafe >= size - min_num_safe:
            #         continue
            #     num_unsafe += 1

            # if safe_next_state:
            #     if num_safe >= min_num_safe:
            #         continue
            #     num_safe += 1
            # if info['rewards']['collision_reward']:
            #     if num_crashed >= min_num_crashed:
            #         continue
            #     num_crashed += 1
            data.append(Transition(torch.Tensor(previous_obs),
                                    torch.Tensor(action),
                                    torch.Tensor(obs),
                                    torch.Tensor([safe_next_state]),
                                    torch.Tensor([reward])))
            previous_obs = obs
            pbar.update(1)

    print("Number of safe states:", num_safe)
    print("Number of crashed states:", num_crashed)
    return data

env = ContinuousRoundaboutEnv(render_mode='rgb_array')
env = RandomNormalNoisyObservation(env)
env.reset()
data = collect_interaction_data(env)
print("Sample transition:", data[0])
print("Number of unsafe states:", int(sum([t.safe_next_state == 0 for t in data])))

100%|██████████| 1000/1000 [00:17<00:00, 56.66it/s]

Number of safe states: 0
Number of crashed states: 0
Sample transition: Transition(state=tensor([[ 1.0000e+00,  1.0019e-02,  1.0000e+00, -9.4366e-05, -9.9907e-02],
        [ 9.9994e-01, -1.6657e-02, -1.0000e+00,  1.9339e-01,  1.1283e-01],
        [ 1.0001e+00, -9.9600e-02, -1.0000e+00,  1.4967e-01,  2.6750e-01],
        [ 9.9996e-01,  6.0565e-01, -9.9999e-01, -1.9421e-01,  1.0010e-01],
        [ 1.0001e+00, -9.5016e-02, -9.9998e-01, -8.2007e-02,  2.3221e-01]]), action=tensor([ 0.0754, -0.2335]), next_state=tensor([[ 9.9992e-01,  5.1357e-04,  1.0000e+00, -3.1193e-02, -1.0000e-01],
        [ 1.0000e+00, -1.1515e-02, -9.9996e-01,  2.5245e-01,  1.3849e-01],
        [ 9.9994e-01,  6.4216e-02, -1.0000e+00,  1.8918e-01, -1.2196e-02],
        [ 1.0001e+00, -9.9069e-02, -1.0001e+00,  2.7606e-02,  2.5547e-01],
        [ 9.9996e-01,  5.3755e-01, -9.9994e-01, -1.6318e-01,  9.9935e-02]]), safe_next_state=tensor([0.]), rewards=tensor([0.]))
Number of unsafe states: 743


Above we have sampled some observations from the environment. This is used to develop a model-based approach to learn about the dynamics of the system, which we will leverage to develop a robust control policy. Now we will set up the data properly and train a Gaussian Process model $$D(s, a) \rightarrow s$$ to predict the dynamics of the system.

In [6]:
from scripts.gp import GPDynamicsModel
from gpytorch.mlls import VariationalELBO, DeepApproximateMLL
BATCH_SIZE = 1

In [7]:
from torch.utils.data import TensorDataset, DataLoader

# append action to flattened state
data_x = []
for t in data:
    state = t.state.flatten()
    data_x.append(torch.cat([state, t.action]))

# flatten state
data_y = []
for t in data:
    data_y.append(t.next_state.flatten())

x_train = torch.stack(data_x)
y_train = torch.stack(data_y)

print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

x_train.shape: torch.Size([1000, 27])
y_train.shape: torch.Size([1000, 25])


In [8]:

from tqdm.notebook import tqdm

model = GPDynamicsModel(input_dims=x_train.shape[-1], output_dims=y_train.shape[-1], num_hidden_dims=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, x_train.shape[-2]))

# Set the number of training iterations
N_EPOCHS = 2

for i in tqdm(range(N_EPOCHS), desc="Epoch"):
    minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        with gpt.settings.num_likelihood_samples(BATCH_SIZE):
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            optimizer.step()

            minibatch_iter.set_postfix(loss=loss.item())

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/1000 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/1000 [00:00<?, ?it/s]

Optionally, let's also test the model to ensure that it is correctly learning the dynamics of the system.

In [47]:
model.eval()
with torch.no_grad():
    mus = []
    variances = []
    lls = []
    minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        preds = model.likelihood(model(x_batch))
        mus.append(preds.mean)
        variances.append(preds.variance)
        lls.append(model.likelihood.log_marginal(y_batch, model(x_batch)))

mus, variances, lls = torch.cat(mus, dim=-1), torch.cat(variances, dim=-1), torch.cat(lls, dim=-1)

rmse = torch.mean(torch.pow(mus.mean() - y_train, 2)).sqrt()
print(f"RMSE: {rmse.item()}, NLL: {-lls.mean().item()}")

Minibatch:   0%|          | 0/1000 [00:00<?, ?it/s]

RMSE: 0.5074037313461304, NLL: 13.076628684997559


In [9]:
class GPModel(object):
    def __init__(self, model):
        self.model = model
        self.model.eval()
        self.model.likelihood.eval()
    
    def predict(self, x):
        x = torch.FloatTensor(x)
        x = x.unsqueeze(0)
        with torch.no_grad():
            preds = self.model(x)
        preds = preds.mean[0]
        # preds = preds.reshape(1, preds.shape[1], 5, 5)
        # preds[:, 0] = torch.where(preds[:, 0] < 0.05, 0, 1)
        preds = preds.reshape(preds.shape[1], 25)

        return preds.numpy()
    
dynamic_model = GPModel(model)

### Cost Function

We must now train a model to determine which states are safe and unsafe. We will use a neural network to learn a cost function $$C(s) \rightarrow c \in \set {0, 1}$$ that will be used to determine the safety of a state. The paper [Constrained Model-based Reinforcement Learning with Robust Cross-Entropy Method](https://arxiv.org/abs/2010.07968) describes a classifier that takes a state as input and determines whether it is safe or unsafe. We will download the associated code that implements a gradient-boosted decision tree classifier to make this prediction.

In [64]:
# !git clone --depth=1 --branch=master https://github.com/liuzuxin/safe-mbrl.git safe-mbrl
# !rm -rf safe-mbrl/.git
# !cp -r safe-mbrl/mbrl mbrl
# !cp -r safe-mbrl/utils utils
# !rm -rf safe-mbrl
# %pip install lightgbm
# %pip install pyyaml
%pip install mpi4py

  Using cached mpi4py-3.1.5.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-macosx_11_0_arm64.whl size=604750 sha256=290d3e992ed54db352f5699e7f58f2d606f3142350d5f8f64b07b195f3828f45
  Stored in directory: /Users/neha/Library/Caches/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py
Note: you may need to restart the kernel to use updated packages.


In [10]:
from scripts.mbrl import CostModel

# CostModel = None

env = ContinuousRoundaboutEnv(render_mode='rgb_array')
env = RandomNormalNoisyObservation(env)
env.reset()

cost_config = {
    'model_param': {
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'max_depth': 5,
        'n_estimators': 100,
        'n_jobs': 1,
        'num_leaves': 12,
    },
    'max_ratio': 7,
    'unsafe_buffer_size': 1000,
    'safe_buffer_size': 500,
    'batch': 10,
    'save': False,
    'save_folder': None,
    'load': False,
    'load_folder': None,
}

cost_model = CostModel(env, obs_size = 25, action_size = 2, idx_goal = 'desired_goal', config=cost_config)

Now we can train the cost model using our previously collected data points.

In [11]:
for t in data:
    state = t.next_state.flatten()
    unsafe = 1 if t.safe_next_state == 0 else 0
    cost_model.add_data_point(state, unsafe)

cost_model.fit()

[LightGBM] [Info] Number of positive: 743, number of negative: 257
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000230 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6375
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.743000 -> initscore=1.061620
[LightGBM] [Info] Start training from score 1.061620
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

## Running the Controller

Now we will run the controller itself using the Safe Model Predictive Control algorithm with sampling derived from the cross-entropy method, outlined in [Constrained Model-based Reinforcement Learning with Robust Cross-Entropy Method](https://arxiv.org/abs/2010.07968).

In [13]:
from scripts.mbrl.controllers import SafeMPC
# from gp import GPModel

mpc_config = {
    'optimizer': "CEM",
    'horizon': 5,
    'gamma': 0.98,
    'CEM': {
        'popsize': 100,
        'max_iters': 7,
        'num_elites': 12,
        'epsilon': 0.001,
        'alpha': 0.2,
        'init_mean': 0,
        'init_var': 1
    }
}

mpc = SafeMPC(env, mpc_config, cost_model=cost_model)

env, display = record_videos(env)

obs, _ = env.reset()
done = False
truncated = False
while not done and not truncated:
    action = mpc.act(dynamic_model, obs.flatten())
    obs, reward, done, truncated, info = env.step(action)

display.stop()
del display
env.close()

show_videos()

/Users/neha/anaconda3/envs/rl/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/neha/Documents/College/f23/CS378/arch/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-1.mp4
Moviepy - Building video /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/neha/Documents/College/f23/CS378/arch/videos/rl-video-episode-0.mp4
